
#IMPORTS DO CÓDIGO


In [10]:
import pandas as pd
import numpy as np
!pip install SQLAlchemy
import sqlalchemy
from sqlalchemy import create_engine, text

In [11]:
import pandas as pd
import numpy as np

def run_etl_pipeline():
    """
    Executa o pipeline completo de ETL para os dados de acidentes de 2019, 2020 e 2021.
    """
    print("--- Iniciando o processo de ETL ---")

    # --- 1. EXTRAÇÃO (Extract) ---
    # Carregando os três conjuntos de dados a partir dos arquivos CSV.
    try:
        print("Etapa 1: Extraindo dados dos arquivos CSV...")
        df_2019 = pd.read_csv('acidentes-2019.csv', delimiter=';')
        df_2020 = pd.read_csv('acidentes_2020-novo.csv', delimiter=';')
        df_2021 = pd.read_csv('acidentes2021.csv', delimiter=';')
        print("Dados extraídos com sucesso!")
    except FileNotFoundError as e:
        print(f"ERRO: Arquivo não encontrado. Verifique se os nomes dos arquivos estão corretos. Detalhes: {e}")
        return

    # --- 2. TRANSFORMAÇÃO (Transform) ---
    print("\nEtapa 2: Transformando os dados...")

    # 2.1 Padronização dos nomes das colunas
    # O dataset de 2019 tem a coluna 'DATA' em maiúsculo. Vamos padronizar para 'data'.
    df_2019.rename(columns={'DATA': 'data'}, inplace=True)
    print("- Nomes de colunas padronizados.")

    # 2.2 Harmonização das colunas (removendo colunas inconsistentes)
    # Colunas presentes em 2019 mas não em 2020/2021
    cols_to_drop_2019 = ['endereco_cruzamento', 'numero_cruzamento', 'referencia_cruzamento', 'descricao']
    df_2019.drop(columns=[col for col in cols_to_drop_2019 if col in df_2019.columns], inplace=True)

    # Coluna 'descricao' presente em 2020 mas não em 2021
    if 'descricao' in df_2020.columns:
        df_2020.drop(columns=['descricao'], inplace=True)

    # A coluna 'tipo' em 2021 está no lugar de 'descricao', mas para unificar vamos manter apenas as colunas em comum.
    # A base de 2021 não tem a coluna 'descricao', então não é necessário fazer nada.

    print("- Colunas harmonizadas entre os datasets.")

    # Lista dos dataframes para aplicar as transformações em lote
    dataframes = [df_2019, df_2020, df_2021]
    ano_inicio = 2019 # Precisa ser alterado caso entre anos anteriores no data set

    # 2.3 Transformação de data e hora
    for i, df in enumerate(dataframes):
        ano = ano_inicio + i
        # Corrigindo valores de hora inválidos como '24:00:00'
        df['hora'] = df['hora'].str.replace('24:00:00', '00:00:00', regex=False)
        # Combinando 'data' e 'hora' em uma única coluna 'timestamp'
        # 'coerce' transforma datas inválidas em NaT (Not a Time)
        df['timestamp'] = pd.to_datetime(df['data'] + ' ' + df['hora'], errors='coerce')
        # Adicionando a coluna 'ano'
        df['ano'] = ano
        # Removendo as colunas originais
        df.drop(columns=['data', 'hora'], inplace=True)

    print("- Colunas de data e hora convertidas para o formato timestamp.")
    print("- Coluna 'ano' criada.")

    # 2.4 Padronização de tipos de dados numéricos
    colunas_numericas = [
        'auto', 'moto', 'ciclom', 'ciclista', 'pedestre', 'onibus',
        'caminhao', 'viatura', 'outros', 'vitimas', 'vitimasfatais'
    ]

    for df in dataframes:
        for col in colunas_numericas:
            if col in df.columns:
                # Converte para numérico, tratando erros, e preenche NaNs com 0
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
                # Converte para inteiro
                df[col] = df[col].astype(int)

    # Tratamento específico para 'num_semaforo' que tem tipos diferentes
    for df in dataframes:
        if 'num_semaforo' in df.columns:
            df['num_semaforo'] = pd.to_numeric(df['num_semaforo'], errors='coerce').fillna(0).astype(int)

    print("- Tipos de dados das colunas numéricas padronizados para inteiro.")

    # Tratamento de endereco
    for df in dataframes:
        if 'endereco' in df.columns:
            df['endereco'] = df['endereco'].str.strip().replace(r'^\s*$', np.nan, regex=True).fillna('Desconhecido')
        if 'detalhe_endereco_acidente' in df.columns:
            df['detalhe_endereco_acidente'] = df['detalhe_endereco_acidente'].str.strip().replace(r'^\s*$', np.nan, regex=True).fillna('Desconhecido')
        if 'complemento' in df.columns:
            df['complemento'] = df['complemento'].str.strip().replace(r'^\s*$', np.nan, regex=True).fillna('Desconhecido')
        if 'numero' in df.columns:
            df['numero'] = pd.to_numeric(df['numero'], errors='coerce').fillna(-1).astype(int)

    print("- Endereços padronizados.")

    # --- 3. CARGA (Load) ---
    print("\nEtapa 3: Carregando os dados transformados...")

    # 3.1 Unificação dos DataFrames
    # Concatenando os três dataframes em um só
    df_unificado = pd.concat(dataframes, ignore_index=True)
    print("- DataFrames unificados com sucesso.")

    # Reordenando colunas para colocar 'ano' e 'timestamp' no início
    cols = ['ano', 'timestamp'] + [col for col in df_unificado.columns if col not in ['ano', 'timestamp']]
    df_unificado = df_unificado[cols]

    # 3.2 Salvando o resultado em um novo arquivo CSV
    output_filename = 'acidentes_unificados_2019-2021.csv'
    df_unificado.to_csv(output_filename, index=False, sep=';', encoding='utf-8')

    print(f"\n--- Processo de ETL Concluído! ---")
    print(f"A base de dados unificada foi salva como: '{output_filename}'")
    print(f"Total de registros na base unificada: {len(df_unificado)}")
    print("\nAmostra dos 5 primeiros registros da base final:")
    print(df_unificado.head())
    print("\nInformações da base final:")
    df_unificado.info()

    # 3.3 Inserindo no banco de dados
    print("\nEtapa 3.3: Inserindo os dados no banco de dados...")

    # criando um arquivo .db
    nome_do_banco_de_dados2 = "banco_etl.db"
    nome_tabela = 'acidentes'

    engine = create_engine(f'sqlite:///{nome_do_banco_de_dados2}')

    df_unificado.to_sql(nome_tabela,
                        con=engine,
                        if_exists='replace',
                        index=False)

    print(f"- Dados inseridos com sucesso na tabela '{nome_tabela}' do banco SQLite (arquivo '{nome_do_banco_de_dados2}').")
# Executando a função principal do pipeline
run_etl_pipeline()


--- Iniciando o processo de ETL ---
Etapa 1: Extraindo dados dos arquivos CSV...
Dados extraídos com sucesso!

Etapa 2: Transformando os dados...
- Nomes de colunas padronizados.
- Colunas harmonizadas entre os datasets.
- Colunas de data e hora convertidas para o formato timestamp.
- Coluna 'ano' criada.
- Tipos de dados das colunas numéricas padronizados para inteiro.
- Endereços padronizados.

Etapa 3: Carregando os dados transformados...
- DataFrames unificados com sucesso.

--- Processo de ETL Concluído! ---
A base de dados unificada foi salva como: 'acidentes_unificados_2019-2021.csv'
Total de registros na base unificada: 18534

Amostra dos 5 primeiros registros da base final:
    ano           timestamp natureza_acidente    situacao       bairro  \
0  2019 2019-01-01 00:41:00        SEM VÍTIMA  FINALIZADA        IPSEP   
1  2019 2019-01-01 01:37:00        SEM VÍTIMA  FINALIZADA   BOA VIAGEM   
2  2019 2019-01-01 14:20:00        SEM VÍTIMA   CANCELADA   BOA VIAGEM   
3  2019 2019

# ETAPA ELT



In [12]:
#INICIANDO A SUBIDA DOS DADOS BRUTOS DE 2019 ATÉ 2021, Utilizando as bibliotecas panda e sqlalchemy

# --- CONFIGURAÇÃO---
# Nome dos arquivos CSV, Importante copiar os arquivos como caminho para executar
lista_arquivos_csv = ['/content/sample_data/Arquivos base de dados/acidentes2019.csv', '/content/sample_data/Arquivos base de dados/acidentes2020.csv', '/content/sample_data/Arquivos base de dados/acidentes2021.csv']

# Nome do arquivo do banco de dados
nome_do_banco_de_dados = 'banco_elt.db'

# Nome da tabela onde os dados brutos serão guardados
nome_da_tabela_consolidada = 'dados_brutos_sinistros'

# Conectar ao banco de dados
engine = create_engine(f'sqlite:///{nome_do_banco_de_dados}')

for i, nome_arquivo in enumerate(lista_arquivos_csv):
  # Lendo os arquivos csv para um dataframe (EXTRACT)
  df_temp  = pd.read_csv(nome_arquivo, sep=';', encoding='utf-8')
  #Adicionar uma coluna para saber a origem do dado
  ano = nome_arquivo.replace('acidentes', '').replace('.csv', '').replace('/content/sample_data/Arquivos base de dados/','')
  df_temp['ano_do_dado'] = int(ano)

  # CARREGAMENTO (LOAD)
  # Define a estratégia: 'replace' para o primeiro arquivo, 'append' para os demais.
  if i == 0:
        modo_de_insercao = 'replace'
  else:
      modo_de_insercao = 'append'

  df_temp.to_sql(
      nome_da_tabela_consolidada,
      con=engine,
      if_exists=modo_de_insercao,
      index=False
  )

#fazendo as transformações dentro do banco
with engine.connect() as conn:

        print("\n🔄 Iniciando transformações no banco de dados (ELT - PLs)...")
        #criar coluna TimeStamp
        conn.execute(text("""
                ALTER TABLE dados_brutos_sinistros ADD COLUMN TimeStamp TEXT;
            """))

        #tratando 2 dados específicos que foram mal prenchidos nos dados de 2019 linhas 8378 e 10215 que serão considerados nulo
        conn.execute(text(f"""
            UPDATE dados_brutos_sinistros
            SET hora = null
            WHERE hora in ("48:00:00","1049592:00:00");
        """))

        #Preencher a coluna timestamp com DATA + hora
        conn.execute(text("""
            UPDATE dados_brutos_sinistros
            SET TimeStamp =
                coalesce(data, DATA) || ' ' ||
                CASE
                    WHEN hora = '24:00:00' THEN '00:00:00'
                    ELSE hora
                END;
        """))
        conn.commit()
        print("- Coluna 'timestamp' preenchida com DATA e hora.")

        #Tratar campo de número, transformando em -1

        conn.execute(text("""
            UPDATE dados_brutos_sinistros
            SET numero = CASE
                WHEN numero IS NULL OR TRIM(numero) = '' THEN -1
                ELSE numero
            END;
        """))
        conn.commit()
        print("- Endereço e número tratados.")

        colunas_numericas = ['auto', 'moto', 'ciclom', 'ciclista', 'pedestre',
                             'onibus', 'caminhao', 'viatura', 'outros',
                             'vitimas', 'vitimasfatais', 'num_semaforo']

        #Padronizando colunas que são númericas, transformando em 0 caso a coluna não exista ou passando para inteiro
        for col in colunas_numericas:
            conn.execute(text(f"""
                UPDATE dados_brutos_sinistros
                SET {col} = CASE
                    WHEN TRIM({col}) = '' OR {col} IS NULL THEN 0
                    ELSE CAST({col} as INTEGER)
                END;
            """))
        conn.commit()
        #Criando coluna de tipo do acidente e preenchendo de acordo com as informações de outras colunas
        conn.execute(text("""
                ALTER TABLE dados_brutos_sinistros ADD COLUMN tipo_acidente TEXT;
            """))
        conn.execute(text("""
            UPDATE dados_brutos_sinistros
            SET tipo_acidente = CASE
                WHEN ciclista > 0 THEN 'Com ciclista'
                WHEN pedestre > 0 THEN 'Com pedestre'
                WHEN vitimasfatais > 0 THEN 'Fatal'
                ELSE 'Outros'
            END;
        """))
        conn.commit()

        #Definindo que caso não haja informação se houveram vítimas, colocar como se não houvessem
        conn.execute(text(f"""
            UPDATE dados_brutos_sinistros
            SET natureza_acidente = 'SEM VÍTIMA'
            WHERE natureza_acidente IS NULL;
        """))

        #criando coluna contendo as colunas dos bancos de dados que são compostas por elementos de texto para substituir por desconhecido no caso de serem vazias ou nulos
        colunas_strings =['bairro','endereco','detalhe_endereco_acidente','complemento','endereco_cruzamento','conservacao_via','ponto_controle','situacao_placa','velocidade_max_via','velocidade_max_via','divisao_via1','divisao_via2','divisao_via3','bairro_cruzamento','tipo','acidente_verificado','tempo_clima','situacao_semaforo','sinalizacao','sentido_via','condicao_via','mao_direcao']
        for col in colunas_strings:
            conn.execute(text(f"""
                UPDATE dados_brutos_sinistros
                SET {col} = COALESCE(NULLIF(TRIM({col}), ''), 'Desconhecido');
            """))
        conn.commit()

        # Adicionar a nova coluna 'id_linha' do tipo INTEGER.
        # Usamos um 'try-except' para o caso de você rodar o código mais de uma vez.
        # Se a coluna já existir, ele apenas informará e continuará.
        conn.execute(text("ALTER TABLE dados_brutos_sinistros ADD COLUMN id_linha INTEGER;"))
        print("- Coluna 'id_linha' adicionada com sucesso.")

        # Passo 2: Preencher a nova coluna com um número de linha único.
        # O ROWID é um identificador interno e sequencial do SQLite.
        conn.execute(text("UPDATE dados_brutos_sinistros SET id_linha = ROWID;"))

        # Confirma a transação
        conn.commit()

        print("- Coluna 'id_linha' preenchida com sucesso.")

        #retirando km/h da velocidade maxima da via, para deixar apenas números
        conn.execute(text("""
            UPDATE dados_brutos_sinistros
            SET velocidade_max_via = CASE
                WHEN velocidade_max_via LIKE '%km/h%' THEN TRIM(REPLACE(velocidade_max_via, 'km/h', ''))
                ELSE velocidade_max_via
            END;
        """))
        conn.commit(text("""
            UPDATE dados_brutos_sinistros
            SET velocidade_max_via = CASE
                WHEN TRIM(velocidade_max_via) GLOB '[0-9]*' THEN CAST(velocidade_max_via AS INTEGER)
                ELSE -1
            END;
        """))

        conn.execute(text)

# Verificação Final
print("--- Verificação Final do Processo ELT ---")

# Contar o total de registros na tabela consolidada
total_registros = pd.read_sql(f"SELECT COUNT(*) FROM {nome_da_tabela_consolidada}", engine).iloc[0,0]
print(f"A tabela final '{nome_da_tabela_consolidada}' contém um total de {total_registros} registros.")

# Agrupar por ano para confirmar que os dados dos 3 arquivos foram carregados
print("\nContagem de registros por ano de origem do dado:")
df_verificacao = pd.read_sql(f'SELECT ano_do_dado, COUNT(*) as total_de_linhas FROM {nome_da_tabela_consolidada} GROUP BY ano_do_dado', engine)

# Mostra a tabela de verificação (no Colab, isso gera a tabela interativa)
from google.colab.data_table import DataTable
DataTable(df_verificacao)


#selecionar todos os dados
df_all = pd.read_sql(f'SELECT * FROM {nome_da_tabela_consolidada} ', engine)
df_all







🔄 Iniciando transformações no banco de dados (ELT - PLs)...
- Coluna 'timestamp' preenchida com DATA e hora.
- Endereço e número tratados.
- Coluna 'id_linha' adicionada com sucesso.
- Coluna 'id_linha' preenchida com sucesso.
--- Verificação Final do Processo ELT ---
A tabela final 'dados_brutos_sinistros' contém um total de 18534 registros.

Contagem de registros por ano de origem do dado:


,DATA,hora,natureza_acidente,situacao,bairro,endereco,numero,detalhe_endereco_acidente,complemento,endereco_cruzamento,...,situacao_placa,velocidade_max_via,mao_direcao,divisao_via1,divisao_via2,divisao_via3,ano_do_dado,TimeStamp,tipo_acidente,id_linha
0,2019-01-01,00:41:00,SEM VÍTIMA,FINALIZADA,IPSEP,AV RECIFE,-1,Desconhecido,LADO OPOSTO AO Nº 3257,AV RECIFE,...,Não há placas,60,Única,Faixa seccionada,Desconhecido,Desconhecido,2019,2019-01-01 00:41:00,Outros,1
1,2019-01-01,01:37:00,SEM VÍTIMA,FINALIZADA,BOA VIAGEM,RUA PADRE BERNADINO PESSOA,-1,RUA MINISTRO NELSON HUNGRIA,Desconhecido,RUA PADRE BERNADINO PESSOA,...,Não há placas,Desconhecido,Única,Não existe,Desconhecido,Desconhecido,2019,2019-01-01 01:37:00,Outros,2
2,2019-01-01,14:20:00,SEM VÍTIMA,CANCELADA,BOA VIAGEM,AV ENGENHEIRO DOMINGOS FERREIRA,-1,RUA DOM JOSE LOPES,"EM FRENTE A DELEGACIA DE BOA VIAGEM, LADO ESQU...",AV ENGENHEIRO DOMINGOS FERREIRA,...,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido,2019,2019-01-01 14:20:00,Outros,3
3,2019-01-01,02:53:00,SEM VÍTIMA,CANCELADA,IMBIRIBEIRA,AV GENERAL MAC ARTHUR,100,RUA JACY,EM FRENTE A ART LED ILUMINAÇÃO,AV GENERAL MAC ARTHUR,...,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido,2019,2019-01-01 02:53:00,Outros,4
4,2019-01-01,08:17:00,COM VÍTIMA,FINALIZADA,JAQUEIRA,RUA TITO ROSAS,63,Desconhecido,ED. JARDINS DA JAQUEIRA,RUA TITO ROSAS,...,Não há placas,40,Única,Faixa seccionada,Desconhecido,Desconhecido,2019,2019-01-01 08:17:00,Com ciclista,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18529,2021-12-31,11:36:00,COM VÍTIMA,FINALIZADA,ESTÂNCIA,AV RECIFE,673,Desconhecido,EM FRENTE AO LABORATORIO CERPE,Desconhecido,...,R-6a,Desconhecido,Dupla,Faixa contínua,Faixa seccionada,Canteiro central,2021,2021-12-31 11:36:00,Outros,18530
18530,2021-12-31,12:56:00,COM VÍTIMA,FINALIZADA,VÁRZEA,RUA GASTAO VIDIGAL,125,Desconhecido,EM FRENTE AO MOTEL DO NINJA,Desconhecido,...,Não há placas,Desconhecido,Dupla,Faixa seccionada,Desconhecido,Desconhecido,2021,2021-12-31 12:56:00,Outros,18531
18531,2021-12-31,15:00:00,COM VÍTIMA,FINALIZADA,TORREÃO,AV NORTE MIGUEL ARRAES DE ALENCAR,-1,RUA NATIVIDADE SALDANHA,ACIDENTE COM VITÍMA NA PRAÇA DA PICANHA / SOLI...,Desconhecido,...,Outras,60,Dupla,Faixa contínua,Faixa seccionada,Canteiro central,2021,2021-12-31 15:00:00,Outros,18532
18532,2021-12-31,20:44:00,COM VÍTIMA,CANCELADA,PRADO,RUA PANDIA CALOGERAS,-1,Desconhecido,PROX AO HABIBIS,Desconhecido,...,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido,Desconhecido,2021,2021-12-31 20:44:00,Outros,18533


In [13]:
# --- CÓDIGO PARA GERAR O ARQUIVO XLSX ---
# (Execute esta célula após ter executado seu código que cria o DataFrame 'df_all')

# Nome que você deseja para o arquivo Excel de saída
nome_do_arquivo_saida = 'dados_brutos_sinistros_consolidados.xlsx'

print(f"Gerando o arquivo Excel: {nome_do_arquivo_saida}...")

df_all.to_excel(
    nome_do_arquivo_saida,
    sheet_name='Dados de Sinistros',
    index=False
)

print(f"✅ Arquivo '{nome_do_arquivo_saida}' gerado com sucesso!")


Gerando o arquivo Excel: dados_brutos_sinistros_consolidados.xlsx...
✅ Arquivo 'dados_brutos_sinistros_consolidados.xlsx' gerado com sucesso!
